In [7]:
import sys
#sys.path.append('./Ray_finrl/findrl_ray/finenv')
from finenv.env_stocktrading import StockTradingEnv
from finenv.preprocessors import FeatureEngineer, data_split
import pandas as pd
import pickle 
import psutil
import ray, time
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total
from ray.tune.registry import register_env
from gymnasium.wrappers import EnvCompatibility
from ray.rllib.agents import ppo
from ray.rllib.algorithms.algorithm import Algorithm

# load the DataFrame from a pickle file
#df = pd.read_pickle('dataset/qqq_proc.pkl')

trade = pd.read_csv('dataset/trade_data.csv')
trade = trade.set_index(trade.columns[0])
trade.index.names = ['']

INDICATORS = ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma']
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
trade.tail()

Stock Dimension: 82, State Space: 821


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
,,,,,,,,,,,,,,,,,,
977,2023-03-29,VRTX,312.549988,316.309998,311.410004,314.239990,1185800.0,2.0,4.272005,316.429879,278.844120,56.805284,201.138085,46.213476,295.763000,299.472166,19.120001,264.841608
977,2023-03-29,WBA,33.980000,34.209999,33.730000,34.150002,6922400.0,2.0,-0.589404,35.703027,31.694973,46.546143,-23.707895,9.421364,34.465000,35.277195,19.120001,264.841608
977,2023-03-29,WDC,35.250000,36.750000,35.250000,36.599998,3704200.0,2.0,-1.045703,38.693062,33.388939,47.458319,-41.506298,1.834005,37.409000,38.855167,19.120001,264.841608
977,2023-03-29,WYNN,108.379997,111.300003,108.379997,109.050003,3011800.0,2.0,0.187761,115.725985,102.293016,57.548050,29.862630,7.383175,108.601667,104.395333,19.120001,264.841608
977,2023-03-29,XEL,65.900002,66.610001,65.790001,66.540001,2450000.0,2.0,-0.144402,67.957722,62.158957,50.356395,48.611903,6.110360,65.277721,67.059890,19.120001,264.841608


In [2]:
#INDICATORS = ['macd','rsi','cci','adx']

In [5]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension
env_kwargs = {
    "hmax": 500,
    "initial_amount": 3000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-2
}
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None, risk_indicator_col= 'vix', **env_kwargs)

In [9]:
def env_creator_trading(env_config):
    # env_config is passed as {} and defaults are set here
    df = env_config.get('df', trade)  # changed from the training environment
    hmax = env_config.get('hmax', 500)
    initial_amount = env_config.get('initial_amount', 3000000)
    num_stock_shares = env_config.get('num_stock_shares', [0] * stock_dimension)
    buy_cost_pct = env_config.get('buy_cost_pct', buy_cost_list)
    sell_cost_pct = env_config.get('sell_cost_pct', sell_cost_list)
    state_space = env_config.get('state_space', 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension)
    stock_dim = env_config.get('stock_dim', stock_dimension)
    tech_indicator_list = env_config.get('tech_indicator_list', INDICATORS)
    action_space = env_config.get('action_space', stock_dimension)
    reward_scaling = env_config.get('reward_scaling', 1e-2)
    return EnvCompatibility(StockTradingEnv(
        df=df,
        hmax=hmax,
        initial_amount=initial_amount,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=buy_cost_pct,
        sell_cost_pct=sell_cost_pct,
        state_space=state_space,
        stock_dim=stock_dim,
        tech_indicator_list=tech_indicator_list,
        action_space=action_space,
        reward_scaling=reward_scaling,
    ))


In [10]:

ray.shutdown()
#ray.init(address='auto')
config = ppo.PPOConfig()
config = config.environment(env_config={'hmax':500,'initial_amount':300000})
config = config.evaluation() 
config = config.resources(num_gpus=0)
config = config.rollouts(num_rollout_workers=0) 
config = config.framework(framework="torch")
config = config.exploration(explore=False)
config['explore'] = False
#config['seed'] = 0
config["model"]["fcnet_hiddens"] = [256, 256, 128,16]

In [11]:
# registering the environment to ray
#trainer = config.build(env="finrl")
#trainer = ppo.PPOTrainer(env='finrl', config=config)
trainer = config.build(env='finrl') 

2023-04-05 11:03:53,536	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


EnvError: The env string you provided ('finrl') is:
a) Not a supported/installed environment.
b) Not a tune-registered environment creator.
c) Not a valid env class string.

Try one of the following:
a) For Atari support: `pip install gym[atari] autorom[accept-rom-license]`.
   For VizDoom support: Install VizDoom
   (https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md) and
   `pip install vizdoomgym`.
   For PyBullet support: `pip install pybullet`.
b) To register your custom env, do `from ray import tune;
   tune.register('[name]', lambda cfg: [return env obj from here using cfg])`.
   Then in your config, do `config['env'] = [name]`.
c) Make sure you provide a fully qualified classpath, e.g.:
   `ray.rllib.examples.env.repeat_after_me_env.RepeatAfterMeEnv`


In [12]:
#load model
#from ray.rllib.algorithms.ppo import PPO
register_env("finrl", env_creator_trading)
cwd_checkpoint = 'model/ppo_new_230405/checkpoint_000000'
#trainer.restore(cwd_checkpoint)
# created in the first place:
trainer = Algorithm.from_checkpoint(cwd_checkpoint)
# change the number of workers if desired
#trainer.restore(cwd_checkpoint)

2023-04-05 11:04:22,033	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
2023-04-05 11:04:22,034	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
/home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")


In [15]:
def DRL_prediction(model, environment):
    start = time.time()
    """make a prediction"""
    state = environment.reset()
    for i in range(len(environment.df.index.unique())):
        action = model.compute_single_action(state, explore=False)
        #print(f'Test Took:{(time.time()-start)}s')
        state, reward, done, _ = environment.step(action)
        if i == (len(environment.df.index.unique()) - 1):
            account_memory = environment.save_asset_memory()
            actions_memory = environment.save_action_memory()
        if done:
            break
    print(f'Test Took:{(time.time()-start)}s')
    return account_memory, actions_memory

In [16]:
df_account_value, df_actions = DRL_prediction(model=trainer, environment = e_trade_gym)
print(df_account_value.iloc[-1]['account_value'])
df_actions.head(100)

Test Took:7.996833562850952s
5102092.2500244165


,AAL,AAPL,ADBE,ADI,ADP,ADSK,ALGN,AMAT,AMD,AMGN,...,TXN,UAL,ULTA,VRSK,VRSN,VRTX,WBA,WDC,WYNN,XEL
date,,,,,,,,,,,,,,,,,,,,,
2019-05-13,0,0,60,6,0,0,44,13,16,0,...,0,0,3,53,0,0,0,8,21,0
2019-05-14,0,0,60,6,0,0,44,13,16,0,...,0,0,3,53,0,0,0,8,21,0
2019-05-15,0,0,60,6,0,0,44,13,16,0,...,0,0,3,53,0,0,0,8,21,0
2019-05-16,0,0,60,6,0,0,44,13,16,0,...,0,0,3,53,0,0,0,8,21,0
2019-05-17,0,0,60,6,0,0,44,13,16,0,...,0,0,3,53,0,0,0,8,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-26,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,-1,0,0
2019-09-27,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,-1,0,0
2019-09-30,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,-1,0,0


In [18]:
for _ in range(3):
    df_account_value, df_actions = DRL_prediction(model=trainer, environment = e_trade_gym)
    print(df_account_value.iloc[-1]['account_value'])

Test Took:8.063332796096802s
5102092.2500244165
Test Took:7.723522424697876s
5102092.2500244165
Test Took:7.92579984664917s
5102092.2500244165


In [ ]:
#Total Trades
counts = (df_actions > 0).sum()
tdays = int(df_actions.shape[0])
print(f'Trades:{counts.sum()}\nTrade Days:{tdays}\nTrades Per Day:{counts.sum()/tdays:.2f}')

In [ ]:
df_actions.sum()

In [ ]:
import datetime
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./results/"+"perf_stats_all"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

In [ ]:
print("==============Compare to NDX===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^NDX', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])